<a href="https://colab.research.google.com/github/Adminixtrator/gpt-2/blob/master/GPT_2_With_SQuAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calling file from Repository




In [1]:
!git clone https://github.com/adminixtrator/gpt-2.git

%cd gpt-2
%ls

Cloning into 'gpt-2'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 126 (delta 1), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (126/126), 13.81 MiB | 18.04 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/home/jovyan/CSE156/gpt-2
Dockerfile.cpu     evaluate-v2.0.py        model_card.md     SQuAD/
Dockerfile.gpu     GPT_2.ipynb             README.md         src/
domains.txt        gpt2_squad.py           requirements.txt  Test_GPT-2.py
download_model.py  GPT_2_With_SQuAD.ipynb  run_squad.py
evaluate-v1.1.py   LICENSE                 setup.py


# Using the gpt-2 model 345M

In [2]:
#Download the gpt-2 model 345M..
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 12.8Mit/s]                                                     
Fetching encoder.json: 1.00kit [00:00, 14.0Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 7.72Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.00kit [00:00, 6.09Mit/s]                                 
Fetching model.ckpt.index: 1.00kit [00:00, 14.2Mit/s]                                               
Fetching model.ckpt.meta: 1.00kit [00:00, 16.3Mit/s]                                                
Fetching vocab.bpe: 1.00kit [00:00, 9.38Mit/s]                                                      


In [3]:
#Encoding..
!export PYTHONIOENCODING=UTF-8

# Now to Implementing gpt-2

In [17]:
# %pwd
!pip install -r /home/jovyan/CSE156/gpt-2/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 128.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.6/601.6 kB 705.5 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for requests==2.21.0 from https://files.pythonhosted.org/packages/7d/e3/20f3d364d6c8e5d2353c72a67778eb189176f08e873c9900e10c0287b84b/requests-2.21.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tqdm==4.31.1 from https://files.pythonhosted.org/packages/6c/4b/c38b5144cf167c4f52288517436ccafefe9dc01b8d1c190e18a6b154cd4a/tqdm-4.31.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/f5/88/eca4e11cca891c46fd75f9a155d76f770d7b687bd298ee15b54d8429da30/boto3-1.34.117-py3-none-any.whl.metadata
  Obtaining dependency information for chardet<3.1.0,>=3.0.2 from https://files.pythonhosted.org/packages/bc/a9/01ffebf

In [21]:
#Changing directory..
import os

os.chdir('/home/jovyan/CSE156/gpt-2/src')


In [5]:
!pip install regex   #For OpenAI GPT

  Obtaining dependency information for regex from https://files.pythonhosted.org/packages/39/29/8158a6e69e97b9c72fab0b46fe4d57c789d07ef91fe4afde23721e7cac61/regex-2024.5.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached regex-2024.5.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached regex-2024.5.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (785 kB)


In [47]:

!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 837.2 kB/s eta 0:00:0000:0100:01


In [57]:
#Importing the necessary libraries..

import json
import numpy as np
import tensorflow as tf
# import model
# import sample
# import encoder

In [58]:
#Function to use the interaction model..

def interact_model(model_name, seed, nsamples, batch_size, length, temperature, top_k, models_dir):
  
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(hparams=hparams, length=length, context=context, batch_size=batch_size, temperature=temperature, top_k=top_k)

        saver = tf.train.Saver(save_relative_paths=True)
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("\nModel prompt >>> ")
            if raw_text == 'ADMIN_NIXTRATOR':
              raw_text = False
              break
            while not raw_text:
                print('\nPrompt should not be empty!')
                raw_text = input("\nModel prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)
            
            

# **Code Explanation**

## **model_name**: 
 This indicates which model we are using. In our case, we are using the GPT-2 model with 345 million parameters or weights

## **seed**: 
Integer seed for random number generators, fix seed to reproduce results

## **nsamples**: 
This represents the number of sample texts generated in our output

## **batch_size**: 
This only affects speed/memory. This must also divide nsamples

*Note: To generate more than one sample, you need to change the values of both nsamples and batch_size and also have to keep them equal.*

## **length**: 
It represents the number of tokens in the generated text. If the length is None, then the number of tokens is decided by model hyperparameters

## **temperature**: 
This controls randomness in Boltzmann distribution. Lower temperature results in less random completions. As the temperature approaches zero, the model will become deterministic and repetitive. Higher temperature results in more random completions

## **top_k**:  
This parameter controls diversity. If the value of top_k is set to 1, this means that only 1 word is considered for each step (token). If top_k is set to 40, that means 40 words are considered at each step. 0 (default) is a special setting meaning no restrictions. top_k = 40 generally is a good value

## **models_dir**: 
It represents the path to parent folder containing model subfolders (contains the <model_name> folder)

# Results

In [59]:
#Using the arguements above..

interact_model('345M', None, 1, 1, 20, 1, 0, '/home/jovyan/CSE156/gpt-2/models')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Fine-tuning on SQuAD for question-answering

In [9]:
#Checking Directory..
os.chdir('/content/gpt-2/SQuAD/')
%ls

create_emb.ipynb  data/  InferSent/  models.py  unsupervised.ipynb


In [ ]:
#Importing the neccessary libraries..
import numpy as np, pandas as pd
import json
import ast 
from textblob import TextBlob
import nltk
import torch
import pickle
from scipy import spatial
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
en_nlp = spacy.load('en')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [ ]:
#Train set 
train = pd.read_json("data/train-v2.0.json")

In [29]:
#Familiarizing with the dataset..
train.shape

(442, 2)

## Loading Embedding dictionary

In [ ]:
def get_target(x):
    idx = -1
    for i in range(len(x["sentences"])):
        if x["text"] in x["sentences"][i]: idx = i
    return idx

In [31]:
train.data

0      {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
1      {'title': 'Frédéric_Chopin', 'paragraphs': [{'...
2      {'title': 'Sino-Tibetan_relations_during_the_M...
3      {'title': 'IPod', 'paragraphs': [{'qas': [{'qu...
4      {'title': 'The_Legend_of_Zelda:_Twilight_Princ...
5      {'title': 'Spectre_(2015_film)', 'paragraphs':...
6      {'title': '2008_Sichuan_earthquake', 'paragrap...
7      {'title': 'New_York_City', 'paragraphs': [{'qa...
8      {'title': 'To_Kill_a_Mockingbird', 'paragraphs...
9      {'title': 'Solar_energy', 'paragraphs': [{'qas...
10     {'title': 'Kanye_West', 'paragraphs': [{'qas':...
11     {'title': 'Buddhism', 'paragraphs': [{'qas': [...
12     {'title': 'American_Idol', 'paragraphs': [{'qa...
13     {'title': 'Dog', 'paragraphs': [{'qas': [{'que...
14     {'title': '2008_Summer_Olympics_torch_relay', ...
15     {'title': 'Genome', 'paragraphs': [{'qas': [{'...
16     {'title': 'Comprehensive_school', 'paragraphs'...
17     {'title': 'Republic_of_t

In [ ]:
train.dropna(inplace=True)

In [33]:
train.shape

(442, 2)

## Data Processing

In [ ]:
def process_data(train):
    
    print("step 1")
    train['sentences'] = train['context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    
    print("step 2")
    train["target"] = train.apply(get_target, axis = 1)
    
    print("step 3")
    train['sent_emb'] = train['sentences'].apply(lambda x: [dict_emb[item][0] if item in\
                                                           dict_emb else np.zeros(4096) for item in x])
    print("step 4")
    train['quest_emb'] = train['question'].apply(lambda x: dict_emb[x] if x in dict_emb else np.zeros(4096) )
        
    return train

In [ ]:

train = process_data(train)

In [ ]:

def cosine_sim(x):
    li = []
    for item in x["sent_emb"]:
        li.append(spatial.distance.cosine(item,x["quest_emb"][0]))
    return li

In [ ]:

def pred_idx(distances):
    return np.argmin(distances)

In [ ]:
#Function to make predictions..
def predictions(train):
    
    train["cosine_sim"] = train.apply(cosine_sim, axis = 1)
    train["diff"] = (train["quest_emb"] - train["sent_emb"])**2
    train["euclidean_dis"] = train["diff"].apply(lambda x: list(np.sum(x, axis = 1)))
    del train["diff"]
    
    print("cosine start")
    
    train["pred_idx_cos"] = train["cosine_sim"].apply(lambda x: pred_idx(x))
    train["pred_idx_euc"] = train["euclidean_dis"].apply(lambda x: pred_idx(x))
    
    return train

In [ ]:
#Making predictions..
predicted = predictions(train)

## Accuracy

In [ ]:
#Function to check accuracy..
def accuracy(target, predicted):
    
    acc = (target==predicted).sum()/len(target)
    
    return acc

In [ ]:
print(accuracy(predicted["target"], predicted["pred_idx_euc"]))    #Accuracy for euclidean Distance

In [ ]:
print(accuracy(predicted["target"], predicted["pred_idx_cos"]))    #Accuracy for Cosine Similarity

## Combed Accuracy

In [ ]:
label = []
for i in range(predicted.shape[0]):
    if predicted.iloc[i,10] == predicted.iloc[i,11]:
        label.append(predicted.iloc[i,10])
    else:
        label.append((predicted.iloc[i,10],predicted.iloc[i,10]))

In [ ]:
ct = 0
for i in range(75206):
    item = predicted["target"][i]
    try:
        if label[i] == predicted["target"][i]: ct +=1
    except:
        if item in label[i]: ct +=1

In [ ]:
ct/75206   #Accuracy..